In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import tensorflow as tf
import hickle as hkl
import random
import scipy.sparse as sp

2025-02-19 19:38:14.841702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-19 19:38:16.025759: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Need to just see how all the datafiles used in DeepCDR looks like

In [3]:
# drug info file - This has the drug ID and the corresponding pubchem ID information
Drug_info_file = pd.read_csv("data/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv")


In [4]:
Drug_info_file.head()

,drug_id,Name,Synonyms,Targets,Target pathway,PubCHEM,Sample Size,Count
0,1242,(5Z)-7-Oxozeaenol,"5Z-7-Oxozeaenol, LL-Z1640-2",TAK1,"Other, kinases",9863776,945,266
1,179,5-Fluorouracil,5-FU,Antimetabolite (DNA & RNA),Other,3385,968,266
2,86,A-443654,KIN001-139,"AKT1, AKT2, AKT3",PI3K/MTOR signaling,10172943,425,266
3,55,A-770041,KIN001-111,"LCK, FYN","Other, kinases",9549184,426,266
4,1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,65110,872,266


In [5]:
# Not yet sure why we need this - May be for TCGA labels? As they denote the cancer types of interest
with open('data/CCLE/Cell_lines_annotations_20181226.txt', 'r') as file:
    Cell_line_info_file = file.read()

In [6]:
# print(Cell_line_info_file)

In [7]:
# Let's look at the graph features file? The contents are in drug_graph_feat folder: Notice their format is .hkl, and we need the package installed inorder to see how these looks like - For the moment, let's just print the contents of the folder.

In [8]:
drug_features_path = 'data/GDSC/drug_graph_feat'

In [9]:
drug_feature_file_list = os.listdir(drug_features_path)

In [10]:
# how many files here?
len(drug_feature_file_list)

223

In [11]:
# print few names of the hkl files
drug_feature_file_list[:3]

['11316960.hkl', '84691.hkl', '6914657.hkl']

In [12]:
# look at the mutation features
Genomic_mutation_features = pd.read_csv('data/CCLE/genomic_mutation_34673_demap_features.csv')

In [13]:
Genomic_mutation_features.shape

(961, 34674)

In [14]:
Genomic_mutation_features.head()

,Unnamed: 0,TNFRSF14.1:2488170,TNFRSF14.1:2489805,TNFRSF14.1:2489824,TNFRSF14.1:2489856,TNFRSF14.1:2489868,TNFRSF14.1:2491328,TNFRSF14.1:2491335,TNFRSF14.1:2491373,TNFRSF14.1:2491403,...,MTCP1.23:154293902,MTCP1.23:154293909,MTCP1.23:154293951,MTCP1.23:154293978,MTCP1.23:154294034,MTCP1.23:154294203,MTCP1.23:154294207,MTCP1.23:154294278,MTCP1.23:154294279,MTCP1.23:154298967
0,ACH-001365,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACH-001364,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACH-001049,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACH-001363,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACH-001362,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# look at gene expressions
Gene_expression_features = pd.read_csv('data/CCLE/genomic_expression_561celllines_697genes_demap_features.csv')

In [16]:
Gene_expression_features.shape

(561, 698)

In [17]:
Gene_expression_features.head()

,Unnamed: 0,LASP1,HOXA11,CREBBP,ETV1,GAS7,CD79B,PAX7,BTK,BRCA1,...,NCKIPSD,MTCP1,DDX3X,FANCG,SSX2,ETV5,CEBPA,LSM14A,CUX1,C15orf65
0,ACH-000828,9.393476,0.042644,3.935460,0.871844,0.070389,0.084064,0.000000,0.056584,3.339137,...,4.071248,3.119356,6.849374,4.355439,0.000000,0.137504,1.769772,6.501598,4.700994,2.295723
1,ACH-000568,7.638074,0.056584,3.427606,0.201634,1.794936,0.739848,0.042644,0.333424,3.193772,...,4.084064,4.634593,5.671576,5.525443,0.056584,2.195348,0.124328,5.811214,3.590961,1.550901
2,ACH-000560,5.728193,6.001352,5.032542,5.018812,0.432959,0.250962,0.000000,0.263034,4.678635,...,5.057450,3.468583,6.617798,6.425761,0.000000,5.203201,1.922198,7.581351,5.320124,1.438293
3,ACH-000561,6.037163,1.565597,4.262283,0.790772,1.257011,0.028569,0.056584,0.042644,3.442280,...,3.400538,3.407353,6.154211,4.794936,0.000000,3.984589,1.028569,6.533719,5.132166,2.144046
4,ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.000000,0.042644,4.939227,...,4.125982,4.047015,6.281884,5.853497,0.056584,3.757023,0.056584,5.912171,4.877744,0.815575


In [18]:
# look at methylation features
Gene_methylation_features = pd.read_csv("data/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv")

In [19]:
Gene_methylation_features.shape

(561, 809)

In [20]:
Gene_methylation_features.head()

,Unnamed: 0,SKI_1_2159133_2160133,TNFRSF14_1_2486803_2487803,PRDM16_1_2984741_2985741,RPL22_1_6259679_6260679,CAMTA1_1_6844383_6845383,MTOR_1_11322608_11323608,PRDM2_1_14025734_14026734,PRDM2_1_14074875_14075875,CASP9_1_15850940_15851940,...,BCORL1_X_129138163_129139163,ELF4_X_129244475_129245475,ELF4_X_129244688_129245688,GPC3_X_133119673_133120673,ATP2B3_X_152800579_152801579,FLNA_X_153603006_153604006,RPL10_X_153625405_153626405,RPL10_X_153625712_153626712,RPL10_X_153625837_153626837,MTCP1_X_154299547_154300547
0,ACH-000698,0.00000,0.00739,0.04799,0.14687,0.00553,0.54138,0.07160,0.00040,0.01836,...,1.00000,0.00000,0.00000,0.020070,0.01706,0.16926,0.00145,0.00242,0.00242,0.01924
1,ACH-000489,0.00000,0.00363,0.64891,1.00000,0.00538,0.00000,0.00000,0.00000,0.00000,...,1.00000,0.00000,0.00000,0.320580,0.72527,0.37572,0.00000,0.00000,0.00000,0.00000
2,ACH-000522,0.00143,0.47465,0.27715,0.19441,0.00858,0.40555,0.21625,0.01367,0.00338,...,1.00000,0.18059,0.22562,0.054325,0.60830,0.65268,0.82345,0.59039,0.59039,0.07546
3,ACH-000613,0.00000,0.00000,0.05112,0.10686,0.00268,0.28618,0.07706,0.00000,0.00000,...,0.01066,0.00000,0.00000,0.010660,0.53086,0.00918,0.00000,0.00448,0.00448,0.00000
4,ACH-000614,0.00118,0.00471,0.15165,0.07407,0.01050,0.51629,0.03292,0.00405,0.00000,...,1.00000,0.07273,0.07273,0.106990,0.69472,0.28947,0.05354,0.04306,0.04306,0.00114


In [21]:
# Also load the response file
response_file_all = pd.read_csv("data/CCLE/GDSC_IC50.csv")

In [22]:
response_file_all.shape

(266, 970)

In [23]:
response_file_all.head() # in columns are the cell line IDs and in rows are the drug IDs.

,Unnamed: 0,ACH-002137,ACH-000474,ACH-002089,ACH-000956,ACH-000948,ACH-000323,ACH-001002,ACH-000905,ACH-000973,...,ACH-002207,ACH-000827,ACH-000534,ACH-001709,ACH-000332,ACH-000469,ACH-000570,ACH-002208,ACH-002317,ACH-000828
0,GDSC:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.724636,2.576109,NaN,NaN,NaN,NaN,2.842845,NaN
1,GDSC:1001,7.258918,9.131374,10.182594,8.332992,8.597157,7.606939,8.168512,8.021503,7.517325,...,5.794405,7.748403,7.899330,6.896593,9.338678,9.200637,6.464226,NaN,9.426281,8.916867
2,GDSC:1004,-3.802467,-5.702659,-4.499864,-4.366928,-4.563040,-4.633045,-6.050113,-6.618734,-5.374042,...,-5.929886,-4.435285,-5.623723,-5.886495,-2.237481,-5.476621,-4.985852,NaN,-1.382754,-2.206867
3,GDSC:1005,4.146364,4.551663,1.772586,3.263056,4.091980,3.090641,1.673430,1.523128,0.794987,...,2.009858,2.576495,3.462176,2.572676,5.706240,2.498858,1.850879,NaN,4.143535,5.344525
4,GDSC:1006,3.171367,-2.014854,-1.818771,0.252367,-1.688469,0.020116,-0.460778,-0.877788,-0.103689,...,-0.121731,-0.889294,0.681844,-2.041620,1.454613,-1.346782,-1.255548,NaN,2.067330,1.230528


In [24]:
response_file_all.shape

(266, 970)

In [25]:
response_file_all.isna().sum().sum()

49020

In [26]:
266*970 - 49020

209000

In [27]:
# Some basic operations to create some data as in the function in DeepCDR

# convert the drug_id --> pubchem_id -  we need this mapping, I think mainly for the SMILES data
reader = csv.reader(open('data/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv','r'))
rows = [item for item in reader]
drugid2pubchemid = {item[0]:item[5] for item in rows if item[5].isdigit()}

In [28]:
len(rows)

267

In [29]:
len(drugid2pubchemid.keys()), len(drugid2pubchemid.values())

(238, 238)

In [30]:
list(drugid2pubchemid.items())[:3] # Note this denote the key (drug ID), and value (pubchem ID) pairs.

[('1242', '9863776'), ('179', '3385'), ('86', '10172943')]

In [31]:
# cellline and cancer type, need to extract the second column (depmap IDs) and the last column (TCGA cancer type) in a dictionary. Key is the cell line (depmap) ID, values will be the cancer type

In [32]:
TCGA_label_set = ["ALL","BLCA","BRCA","CESC","DLBC","LIHC","LUAD",
                  "ESCA","GBM","HNSC","KIRC","LAML","LCML","LGG",
                  "LUSC","MESO","MM","NB","OV","PAAD","SCLC","SKCM",
                  "STAD","THCA",'COAD/READ']

In [33]:
Cell_line_info_file_path = 'data/CCLE/Cell_lines_annotations_20181226.txt'

In [34]:
#map cellline --> cancer type
cellline2cancertype ={}
for line in open(Cell_line_info_file_path).readlines()[1:]:
    cellline_id = line.split('\t')[1]
    TCGA_label = line.strip().split('\t')[-1]
    #if TCGA_label in TCGA_label_set: Does not look like it does this check here though
    cellline2cancertype[cellline_id] = TCGA_label

In [35]:
# cellline2cancertype

In [36]:
#load demap cell lines genomic mutation features
mutation_feature = pd.read_csv('data/CCLE/genomic_mutation_34673_demap_features.csv',sep=',',header=0,index_col=[0])
cell_line_id_set = list(mutation_feature.index)

In [37]:
mutation_feature.shape

(961, 34673)

In [38]:
len(cell_line_id_set)

961

In [39]:
Drug_feature_file = 'data/GDSC/drug_graph_feat'
# The hickle files are stored according to the Pubchem IDs

In [40]:
# Okay, now to loading the hickle files
drug_pubchem_id_set = []
drug_feature = {}
for each in os.listdir(Drug_feature_file):
    drug_pubchem_id_set.append(each.split('.')[0])
    feat_mat,adj_list,degree_list = hkl.load('%s/%s'%(Drug_feature_file,each))
    drug_feature[each.split('.')[0]] = [feat_mat,adj_list,degree_list]

/common/statsgeneral/gayara/cdr_env_2025/lib/python3.8/site-packages/hickle/hickle.py:356: UserWarning: Input argument 'file_obj' appears to be a file made with hickle v3. Using legacy load...
  warnings.warn("Input argument 'file_obj' appears to be a file made"


In [41]:
# just select a single file here from the loaded hickle files

# We have the pubchem ID as the key in the dictionary, and three features related to the drug: Feature matrix, adjacency matrix and the degree list
pubchem_ID, drug_related_vals = list(drug_feature.items())[0]

In [42]:
drug_features = drug_related_vals[0]
drug_features

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [43]:
drug_features.shape

(34, 75)

In [44]:
# gives a list of adajecency atoms of the drug
adjecency_matrix = drug_related_vals[1]

In [45]:
len(adjecency_matrix)

34

In [46]:
# degree list - specifies the number of neighbors for each atom
degree_info = drug_related_vals[-1]
len(degree_info)

34

In [47]:
print(len(drug_pubchem_id_set)==len(drug_feature.values()))

True


In [48]:
#load gene expression faetures
gexpr_feature = pd.read_csv('data/CCLE/genomic_expression_561celllines_697genes_demap_features.csv',sep=',',header=0,index_col=[0])
    
#only keep overlapped cell lines
mutation_feature = mutation_feature.loc[list(gexpr_feature.index)]

In [49]:
gexpr_feature.shape, mutation_feature.shape

((561, 697), (561, 34673))

In [50]:
#load methylation 
methylation_feature = pd.read_csv('data/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv',sep=',',header=0,index_col=[0])


In [51]:
methylation_feature.shape

(561, 808)

In [52]:
methylation_feature.shape, gexpr_feature.shape, mutation_feature.shape

((561, 808), (561, 697), (561, 34673))

In [53]:
# All omics have the same shape - we have data for the same cell line IDs

In [54]:
experiment_data = pd.read_csv('data/CCLE/GDSC_IC50.csv',sep=',',header=0,index_col=[0])

In [55]:
experiment_data.shape

(266, 969)

In [56]:
#filter experiment data
drug_match_list=[item for item in experiment_data.index if item.split(':')[1] in drugid2pubchemid.keys()]
experiment_data_filtered = experiment_data.loc[drug_match_list]

In [57]:
experiment_data_filtered.shape

(238, 969)

In [58]:
experiment_data_filtered.head()

,ACH-002137,ACH-000474,ACH-002089,ACH-000956,ACH-000948,ACH-000323,ACH-001002,ACH-000905,ACH-000973,ACH-000896,...,ACH-002207,ACH-000827,ACH-000534,ACH-001709,ACH-000332,ACH-000469,ACH-000570,ACH-002208,ACH-002317,ACH-000828
GDSC:1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.724636,2.576109,NaN,NaN,NaN,NaN,2.842845,NaN
GDSC:1001,7.258918,9.131374,10.182594,8.332992,8.597157,7.606939,8.168512,8.021503,7.517325,7.215900,...,5.794405,7.748403,7.899330,6.896593,9.338678,9.200637,6.464226,NaN,9.426281,8.916867
GDSC:1004,-3.802467,-5.702659,-4.499864,-4.366928,-4.563040,-4.633045,-6.050113,-6.618734,-5.374042,-3.826083,...,-5.929886,-4.435285,-5.623723,-5.886495,-2.237481,-5.476621,-4.985852,NaN,-1.382754,-2.206867
GDSC:1005,4.146364,4.551663,1.772586,3.263056,4.091980,3.090641,1.673430,1.523128,0.794987,2.256727,...,2.009858,2.576495,3.462176,2.572676,5.706240,2.498858,1.850879,NaN,4.143535,5.344525
GDSC:1006,3.171367,-2.014854,-1.818771,0.252367,-1.688469,0.020116,-0.460778,-0.877788,-0.103689,0.021624,...,-0.121731,-0.889294,0.681844,-2.041620,1.454613,-1.346782,-1.255548,NaN,2.067330,1.230528


In [59]:
data_idx = []
for each_drug in experiment_data_filtered.index:
    for each_cellline in experiment_data_filtered.columns:
        pubchem_id = drugid2pubchemid[each_drug.split(':')[-1]]
        if str(pubchem_id) in drug_pubchem_id_set and each_cellline in mutation_feature.index:
            if not np.isnan(experiment_data_filtered.loc[each_drug,each_cellline]) and each_cellline in cellline2cancertype.keys():
                ln_IC50 = float(experiment_data_filtered.loc[each_drug,each_cellline])
                data_idx.append((each_cellline,pubchem_id,ln_IC50,cellline2cancertype[each_cellline])) 
nb_celllines = len(set([item[0] for item in data_idx]))
nb_drugs = len(set([item[1] for item in data_idx]))

In [60]:
len(data_idx)

107446

In [61]:
data_idx[0]

('ACH-000070', '176870', 0.693305, 'ALL')

In [62]:
# data index compose of the cell_line ID, pubchem_ID, response, cancer_type

In [63]:
nb_celllines, nb_drugs

(561, 223)

In [64]:
mutation_feature.shape, methylation_feature.shape, gexpr_feature.shape

((561, 34673), (561, 808), (561, 697))

In [65]:
len(drug_feature)

223

In [66]:
# This will create the train and test lists - with data index lists that contain cellline ID, pubchem ID, response and TCGA label
# Also over here we check for the cancer type, and only retain the samples if their TCGA label is in the chosen cancer types list specified in TCGA_label_set
def DataSplit(data_idx,ratio = 0.95):
    data_train_idx,data_test_idx = [], []
    for each_type in TCGA_label_set:
        data_subtype_idx = [item for item in data_idx if item[-1]==each_type]
        train_list = random.sample(data_subtype_idx,int(ratio*len(data_subtype_idx)))
        test_list = [item for item in data_subtype_idx if item not in train_list]
        data_train_idx += train_list
        data_test_idx += test_list
    return data_train_idx,data_test_idx

In [67]:
data_train_idx, data_test_idx = DataSplit(data_idx,ratio = 0.95)

In [68]:
len(data_train_idx), len(data_test_idx)

(89585, 4729)

In [69]:
# how many final samples?
len(data_train_idx) + len(data_test_idx)

94314

In [70]:
# see if these have the same data types as the data_idx list
data_train_idx[0]

('ACH-000953', '24826799', -2.253904, 'ALL')

In [71]:
data_test_idx[0]

('ACH-000960', '84691', 2.16617, 'ALL')

In [72]:
def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    # the following line does the operation (AD)'D = D'A'D
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

In [73]:
# notice that the adjacency matrix that we extract from the provided hickle files is a list. I think this normalization refers to the final 100*100 adjacency matrix.

In [74]:
generate_adj_mat = np.array([[5,5],[3,2]])

In [75]:
generate_adj_mat.shape

(2, 2)

In [76]:
generate_adj_mat

array([[5, 5],
       [3, 2]])

In [77]:
norm_mat_for_this = NormalizeAdj(generate_adj_mat)

In [78]:
norm_mat_for_this

array([[0.54545455, 0.36927447],
       [0.61545745, 0.5       ]])

In [79]:
generate_adj_mat = generate_adj_mat + np.eye(generate_adj_mat.shape[0])

In [80]:
generate_adj_mat

array([[6., 5.],
       [3., 3.]])

In [81]:
generate_adj_mat.sum(1)

array([11.,  6.])

In [82]:
np.array(generate_adj_mat.sum(1))

array([11.,  6.])

In [83]:
np.power(np.array(generate_adj_mat.sum(1)), -0.5)

array([0.30151134, 0.40824829])

In [84]:
np.power(np.array(generate_adj_mat.sum(1)), -0.5).flatten()

array([0.30151134, 0.40824829])

In [85]:
11**(-0.5), 6**(-0.5)

(0.30151134457776363, 0.408248290463863)

In [86]:
d_trial = sp.diags(np.power(np.array(generate_adj_mat.sum(1)), -0.5).flatten(), 0).toarray()

In [87]:
d_trial

array([[0.30151134, 0.        ],
       [0.        , 0.40824829]])

In [88]:
generate_adj_mat.dot(d_trial)

array([[1.80906807, 2.04124145],
       [0.90453403, 1.22474487]])

In [89]:
# quick verify
6*0.30151134, 5*0.40824829, 3*0.30151134, 3*0.40824829

(1.80906804, 2.0412414500000002, 0.90453402, 1.22474487)

In [90]:
generate_adj_mat.dot(d_trial).transpose()

array([[1.80906807, 0.90453403],
       [2.04124145, 1.22474487]])

In [91]:
generate_adj_mat.dot(d_trial).transpose().dot(d_trial)

array([[0.54545455, 0.36927447],
       [0.61545745, 0.5       ]])

In [92]:
1.80906807*0.30151134,  0.90453403*0.40824829, 2.04124145*0.30151134, 1.22474487*0.40824829

(0.5454545379369138,
 0.3692744709943087,
 0.615457444853043,
 0.49999999886377233)

In [93]:
a_norm_trial = generate_adj_mat.dot(d_trial).transpose().dot(d_trial)

In [94]:
a_norm_trial

array([[0.54545455, 0.36927447],
       [0.61545745, 0.5       ]])

In [95]:
# This is creating a random matrix for adjacency matrix: I believe this is for the additional part that we append to the adjacency matrix: The output from here is a symmetric matrix.
def random_adjacency_matrix(n):   
    matrix = [[random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself - therefore the diagonals are set to 0s.
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return matrix

In [96]:
# suppose n = 10 here
random_int_list = [random.randint(0, 1) for i in range(10)]

In [97]:
random_int_list

[0, 1, 1, 1, 0, 0, 1, 0, 1, 1]

In [98]:
matrix = [[random.randint(0, 1) for i in range(10)] for j in range(10)]

In [99]:
len(matrix)

10

In [100]:
matrix

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 1, 1]]

In [101]:
for i in range(10):
    matrix[i][i] = 0

In [102]:
matrix

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 1, 1, 0],
 [1, 1, 0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 1, 0]]

In [103]:
for i in range(10):
    for j in range(10):
        matrix[j][i] = matrix[i][j]

In [104]:
matrix

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 1, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]]

In [105]:
# now this where all the final features abd adjacency matrix are produced

In [106]:
Max_atoms = 100

In [107]:
def CalculateGraphFeat(feat_mat,adj_list):
    assert feat_mat.shape[0] == len(adj_list)
    feat = np.zeros((Max_atoms,feat_mat.shape[-1]),dtype='float32')
    adj_mat = np.zeros((Max_atoms,Max_atoms),dtype='float32')
    if israndom:
        feat = np.random.rand(Max_atoms,feat_mat.shape[-1])
        adj_mat[feat_mat.shape[0]:,feat_mat.shape[0]:] = random_adjacency_matrix(Max_atoms-feat_mat.shape[0])        
    feat[:feat_mat.shape[0],:] = feat_mat
    for i in range(len(adj_list)):
        nodes = adj_list[i]
        for each in nodes:
            adj_mat[i,int(each)] = 1
    assert np.allclose(adj_mat,adj_mat.T)
    adj_ = adj_mat[:len(adj_list),:len(adj_list)]
    adj_2 = adj_mat[len(adj_list):,len(adj_list):]
    norm_adj_ = NormalizeAdj(adj_)
    norm_adj_2 = NormalizeAdj(adj_2)
    adj_mat[:len(adj_list),:len(adj_list)] = norm_adj_
    adj_mat[len(adj_list):,len(adj_list):] = norm_adj_2    
    return [feat,adj_mat]

In [108]:
drug_features.shape[0] == len(adjecency_matrix)

True

In [109]:
len(adjecency_matrix)

34

In [110]:
drug_features.shape

(34, 75)

In [111]:
feat_T = np.zeros((Max_atoms,drug_features.shape[-1]),dtype='float32')

In [112]:
np.mean(feat_T)

0.0

In [113]:
feat_T[:drug_features.shape[0],:] = drug_features

In [114]:
drug_features.shape

(34, 75)

In [115]:
feat_T.shape

(100, 75)

In [116]:
# Check if the indices corresponding to the features in the featuremap matches with the final feat_T matrix

In [117]:
np.mean(feat_T[:34, :] == drug_features)

1.0

In [118]:
# check if all the other means are 0
np.mean(feat_T[34:, :])

0.0

In [119]:
# Notice that this happens in the function only if the israndom variable is set to False (which I think the paper is talking about)

# Else we can also go with appending the matrix size to Max_atoms*features with the defined random adjacency matrix. This also changes the feature matrix (I mean setting israndom to False)

In [120]:
# In that case feature map becomes
feat_alt = np.random.rand(Max_atoms,drug_features.shape[-1])
feat_alt[:drug_features.shape[0],:] = drug_features

In [121]:
feat_alt

array([[1.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.36076194, 0.79358874, 0.11329305, ..., 0.29797569, 0.83877069,
        0.51280729],
       [0.47434159, 0.68817354, 0.75768153, ..., 0.83383537, 0.56905275,
        0.70947795],
       [0.21879443, 0.98399148, 0.0296639 , ..., 0.80380846, 0.92703749,
        0.64302065]])

In [122]:
feat_alt.shape

(100, 75)

In [123]:
# Basically, this will have the first 34 rows (in this examples) extactly as the features given in the hickle file. The rest of the rows will be filled with random uniform values instead of zeros as with israndom = False case

In [124]:
# Let's see later what will happen to the adjacency matrix if we set isradom to True later

In [125]:
# now what when israndom is set to False: We proceed as normal

In [126]:
adj_mat_T= np.zeros((Max_atoms,Max_atoms),dtype='float32')

In [127]:
adj_mat_T.shape, np.mean(adj_mat_T)

((100, 100), 0.0)

In [128]:
# our adjacency list is given by adjecency_matrix and mind that this is a list
len(adjecency_matrix)

34

In [129]:
# Let's create the adjacency matrix here - Basically tells to connect each node with whatever their neighbors are as specified in the adjacency list

In [130]:
for i in range(len(adjecency_matrix)):
    nodes = adjecency_matrix[i]
    for each in nodes:
        adj_mat_T[i,int(each)] = 1

In [131]:
# This works, as now the mean of this matrix is not zeor
np.mean(adj_mat_T)

0.0076

In [132]:
# but does not seem like we are done here - Let's follow through with the rest of the code -  basically we are using the normalizing adjacency matrix function defined earlier to normalize the adjacency matrix

In [133]:
# This np.allclose besically see if two arrays are roughly simialr - close enough
np.allclose(adj_mat_T,adj_mat_T.T)

True

In [134]:
adj_ = adj_mat_T[:len(adjecency_matrix),:len(adjecency_matrix)]
adj_2 = adj_mat_T[len(adjecency_matrix):,len(adjecency_matrix):]
norm_adj_ = NormalizeAdj(adj_)
norm_adj_2 = NormalizeAdj(adj_2)
adj_mat_T[:len(adjecency_matrix),:len(adjecency_matrix)] = norm_adj_
adj_mat_T[len(adjecency_matrix):,len(adjecency_matrix):] = norm_adj_2 

In [135]:
adj_.shape, adj_2.shape

((34, 34), (66, 66))

In [136]:
np.mean(adj_), np.mean(adj_2)

(0.029224653, 0.015151516)

In [137]:
norm_adj_.shape, norm_adj_2.shape

((34, 34), (66, 66))

In [138]:
np.mean(norm_adj_), np.mean(norm_adj_2)

(0.029224652760585675, 0.015151515151515152)

In [139]:
adj_mat_T.shape

(100, 100)

In [140]:
# now this last thing is going to be your adjacency matrix

In [141]:
np.mean(adj_mat_T)

0.00997837

In [142]:
# now what if we set is random to True? What is the deal with the adjacency matrix?

In [143]:
# This is the line that is going to be affected
# adj_mat[feat_mat.shape[0]:,feat_mat.shape[0]:] = random_adjacency_matrix(Max_atoms-feat_mat.shape[0])        

In [144]:
# Let's define the adjacency matrix with all zeros again
adj_mat_T= np.zeros((Max_atoms,Max_atoms),dtype='float32')

In [145]:
adj_mat_T.shape

(100, 100)

In [146]:
adj_mat_T[drug_features.shape[0]:,drug_features.shape[0]:] = random_adjacency_matrix(Max_atoms-drug_features.shape[0])  

In [147]:
len(random_adjacency_matrix(Max_atoms-drug_features.shape[0]))

66

In [148]:
adj_mat_T[drug_features.shape[0]:,drug_features.shape[0]:].shape

(66, 66)

In [149]:
adj_mat_T # basically the last quadrant of the matrix of shape 66*66 is set to the random adjecency matrix define earlier if israndom is set to True

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]], dtype=float32)

In [150]:
# Moving on to the next function
def FeatureExtract(data_idx,drug_feature,mutation_feature,gexpr_feature,methylation_feature):
    cancer_type_list = []
    nb_instance = len(data_idx)
    nb_mutation_feature = mutation_feature.shape[1]
    nb_gexpr_features = gexpr_feature.shape[1]
    nb_methylation_features = methylation_feature.shape[1]
    drug_data = [[] for item in range(nb_instance)]
    mutation_data = np.zeros((nb_instance,1, nb_mutation_feature,1),dtype='float32')
    gexpr_data = np.zeros((nb_instance,nb_gexpr_features),dtype='float32') 
    methylation_data = np.zeros((nb_instance, nb_methylation_features),dtype='float32') 
    target = np.zeros(nb_instance,dtype='float32')
    for idx in range(nb_instance):
        cell_line_id,pubchem_id,ln_IC50,cancer_type = data_idx[idx]
        #modify
        feat_mat,adj_list,_ = drug_feature[str(pubchem_id)]
        #fill drug data,padding to the same size with zeros
        drug_data[idx] = CalculateGraphFeat(feat_mat,adj_list)
        #randomlize X A
        mutation_data[idx,0,:,0] = mutation_feature.loc[cell_line_id].values
        gexpr_data[idx,:] = gexpr_feature.loc[cell_line_id].values
        methylation_data[idx,:] = methylation_feature.loc[cell_line_id].values
        target[idx] = ln_IC50
        cancer_type_list.append([cancer_type,cell_line_id,pubchem_id])
    return drug_data,mutation_data,gexpr_data,methylation_data,target,cancer_type_list

In [151]:
# should all be 
561*223

125103

In [152]:
107440

107440

In [153]:
900*266

239400

In [154]:
# Let's make a dataframe with all drug IDs, cell line IDs, features, and responses

In [155]:
# We have the cell ID, pubchem ID, response and a TCGA_label in data_idx

In [156]:
len(data_idx)

107446

In [157]:
# should we make this a dataframe first? Or maybe better to get rid of the cancer types not considered

In [158]:
all_req_data_idx = []
for item in data_idx:
    if item[-1] in TCGA_label_set:
        all_req_data_idx.append(item)

In [159]:
len(all_req_data_idx)

94314

In [160]:
# verify this number is correct
len(data_train_idx) + len(data_test_idx)

94314

In [161]:
# convert this list now to a dataframe
DeepCDR_data = pd.DataFrame(all_req_data_idx, columns = ["Cell_line_ID", "Pubchem_ID", "Response", "TCGA_cancer_type"])

In [162]:
DeepCDR_data.head()

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type
0,ACH-000070,176870,0.693305,ALL
1,ACH-000137,176870,2.580268,GBM
2,ACH-000008,176870,2.557837,SKCM
3,ACH-000740,176870,0.290013,HNSC
4,ACH-000697,176870,1.110250,DLBC


In [163]:
DeepCDR_data.shape

(94314, 4)

In [164]:
# how many unique cell line IDS? - 494
DeepCDR_data['Cell_line_ID'].value_counts()

ACH-000944    238
ACH-000204    238
ACH-000007    238
ACH-000371    238
ACH-000743    238
             ... 
ACH-000763     59
ACH-000187     51
ACH-000225     29
ACH-000035     28
ACH-000345     25
Name: Cell_line_ID, Length: 494, dtype: int64

In [165]:
len(DeepCDR_data['Cell_line_ID'].value_counts())

494

In [166]:
# how many unique pubchem IDs? - 223
DeepCDR_data['Pubchem_ID'].value_counts()

46224516    924
46883536    920
46907787    904
10196499    901
9956119     900
           ... 
5494449     172
11844351    172
49836027    171
176870      161
5384616     155
Name: Pubchem_ID, Length: 223, dtype: int64

In [167]:
len(DeepCDR_data['Pubchem_ID'].value_counts())

223

In [168]:
# now, I think we can just go ahead and 

In [169]:
# convert the cell line IDs to a list, this maybe useful when merging the values
cell_line_ID_list = DeepCDR_data['Cell_line_ID'].to_list()

In [170]:
# here are the multi-omics data ypes
methylation_feature.shape, gexpr_feature.shape, mutation_feature.shape

((561, 808), (561, 697), (561, 34673))

In [171]:
# For all these datasets the index column is the cell line ID, therefore reset the index

In [172]:
methylation_feature_alt = methylation_feature.reset_index()

In [173]:
methylation_feature_alt = methylation_feature_alt.rename(columns = {'index': 'Cell_line_ID'})

In [174]:
methylation_feature_alt.head()

,Cell_line_ID,SKI_1_2159133_2160133,TNFRSF14_1_2486803_2487803,PRDM16_1_2984741_2985741,RPL22_1_6259679_6260679,CAMTA1_1_6844383_6845383,MTOR_1_11322608_11323608,PRDM2_1_14025734_14026734,PRDM2_1_14074875_14075875,CASP9_1_15850940_15851940,...,BCORL1_X_129138163_129139163,ELF4_X_129244475_129245475,ELF4_X_129244688_129245688,GPC3_X_133119673_133120673,ATP2B3_X_152800579_152801579,FLNA_X_153603006_153604006,RPL10_X_153625405_153626405,RPL10_X_153625712_153626712,RPL10_X_153625837_153626837,MTCP1_X_154299547_154300547
0,ACH-000698,0.00000,0.00739,0.04799,0.14687,0.00553,0.54138,0.07160,0.00040,0.01836,...,1.00000,0.00000,0.00000,0.020070,0.01706,0.16926,0.00145,0.00242,0.00242,0.01924
1,ACH-000489,0.00000,0.00363,0.64891,1.00000,0.00538,0.00000,0.00000,0.00000,0.00000,...,1.00000,0.00000,0.00000,0.320580,0.72527,0.37572,0.00000,0.00000,0.00000,0.00000
2,ACH-000522,0.00143,0.47465,0.27715,0.19441,0.00858,0.40555,0.21625,0.01367,0.00338,...,1.00000,0.18059,0.22562,0.054325,0.60830,0.65268,0.82345,0.59039,0.59039,0.07546
3,ACH-000613,0.00000,0.00000,0.05112,0.10686,0.00268,0.28618,0.07706,0.00000,0.00000,...,0.01066,0.00000,0.00000,0.010660,0.53086,0.00918,0.00000,0.00448,0.00448,0.00000
4,ACH-000614,0.00118,0.00471,0.15165,0.07407,0.01050,0.51629,0.03292,0.00405,0.00000,...,1.00000,0.07273,0.07273,0.106990,0.69472,0.28947,0.05354,0.04306,0.04306,0.00114


In [175]:
methylation_feature_alt.shape

(561, 809)

In [176]:
# add a suffix '_methyl' to all columns in the dataset
colnames_with_methyl = [col_name + '_methyl' for col_name in methylation_feature_alt.columns[1:]]
cell_line_col = ['Cell_line_ID']

In [177]:
len(colnames_with_methyl)

808

In [178]:
all_methyl_columns = cell_line_col + colnames_with_methyl

In [179]:
# all_methyl_columns

In [180]:
methylation_feature_alt.columns = all_methyl_columns

In [181]:
methylation_feature_alt.head()

,Cell_line_ID,SKI_1_2159133_2160133_methyl,TNFRSF14_1_2486803_2487803_methyl,PRDM16_1_2984741_2985741_methyl,RPL22_1_6259679_6260679_methyl,CAMTA1_1_6844383_6845383_methyl,MTOR_1_11322608_11323608_methyl,PRDM2_1_14025734_14026734_methyl,PRDM2_1_14074875_14075875_methyl,CASP9_1_15850940_15851940_methyl,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
0,ACH-000698,0.00000,0.00739,0.04799,0.14687,0.00553,0.54138,0.07160,0.00040,0.01836,...,1.00000,0.00000,0.00000,0.020070,0.01706,0.16926,0.00145,0.00242,0.00242,0.01924
1,ACH-000489,0.00000,0.00363,0.64891,1.00000,0.00538,0.00000,0.00000,0.00000,0.00000,...,1.00000,0.00000,0.00000,0.320580,0.72527,0.37572,0.00000,0.00000,0.00000,0.00000
2,ACH-000522,0.00143,0.47465,0.27715,0.19441,0.00858,0.40555,0.21625,0.01367,0.00338,...,1.00000,0.18059,0.22562,0.054325,0.60830,0.65268,0.82345,0.59039,0.59039,0.07546
3,ACH-000613,0.00000,0.00000,0.05112,0.10686,0.00268,0.28618,0.07706,0.00000,0.00000,...,0.01066,0.00000,0.00000,0.010660,0.53086,0.00918,0.00000,0.00448,0.00448,0.00000
4,ACH-000614,0.00118,0.00471,0.15165,0.07407,0.01050,0.51629,0.03292,0.00405,0.00000,...,1.00000,0.07273,0.07273,0.106990,0.69472,0.28947,0.05354,0.04306,0.04306,0.00114


In [182]:
# maybe do the same for the other two datasets, then merge these three together, and the merge with the main dataset at the end


In [183]:
gexpr_feature_alt = gexpr_feature.reset_index()

In [184]:
gexpr_feature_alt = gexpr_feature_alt.rename(columns = {'index': 'Cell_line_ID'})

In [185]:
gexpr_feature_alt.head()

,Cell_line_ID,LASP1,HOXA11,CREBBP,ETV1,GAS7,CD79B,PAX7,BTK,BRCA1,...,NCKIPSD,MTCP1,DDX3X,FANCG,SSX2,ETV5,CEBPA,LSM14A,CUX1,C15orf65
0,ACH-000828,9.393476,0.042644,3.935460,0.871844,0.070389,0.084064,0.000000,0.056584,3.339137,...,4.071248,3.119356,6.849374,4.355439,0.000000,0.137504,1.769772,6.501598,4.700994,2.295723
1,ACH-000568,7.638074,0.056584,3.427606,0.201634,1.794936,0.739848,0.042644,0.333424,3.193772,...,4.084064,4.634593,5.671576,5.525443,0.056584,2.195348,0.124328,5.811214,3.590961,1.550901
2,ACH-000560,5.728193,6.001352,5.032542,5.018812,0.432959,0.250962,0.000000,0.263034,4.678635,...,5.057450,3.468583,6.617798,6.425761,0.000000,5.203201,1.922198,7.581351,5.320124,1.438293
3,ACH-000561,6.037163,1.565597,4.262283,0.790772,1.257011,0.028569,0.056584,0.042644,3.442280,...,3.400538,3.407353,6.154211,4.794936,0.000000,3.984589,1.028569,6.533719,5.132166,2.144046
4,ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.000000,0.042644,4.939227,...,4.125982,4.047015,6.281884,5.853497,0.056584,3.757023,0.056584,5.912171,4.877744,0.815575


In [186]:
# add a suffix '_gexpr' to all columns in the dataset
colnames_with_expression = [col_name + '_gexpr' for col_name in gexpr_feature_alt.columns[1:]]
cell_line_col_expr = ['Cell_line_ID']

In [187]:
all_gexpr_cols = cell_line_col_expr + colnames_with_expression

In [188]:
len(all_gexpr_cols)

698

In [189]:
gexpr_feature_alt.columns = all_gexpr_cols

In [190]:
gexpr_feature_alt

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,NCKIPSD_gexpr,MTCP1_gexpr,DDX3X_gexpr,FANCG_gexpr,SSX2_gexpr,ETV5_gexpr,CEBPA_gexpr,LSM14A_gexpr,CUX1_gexpr,C15orf65_gexpr
0,ACH-000828,9.393476,0.042644,3.935460,0.871844,0.070389,0.084064,0.000000,0.056584,3.339137,...,4.071248,3.119356,6.849374,4.355439,0.000000,0.137504,1.769772,6.501598,4.700994,2.295723
1,ACH-000568,7.638074,0.056584,3.427606,0.201634,1.794936,0.739848,0.042644,0.333424,3.193772,...,4.084064,4.634593,5.671576,5.525443,0.056584,2.195348,0.124328,5.811214,3.590961,1.550901
2,ACH-000560,5.728193,6.001352,5.032542,5.018812,0.432959,0.250962,0.000000,0.263034,4.678635,...,5.057450,3.468583,6.617798,6.425761,0.000000,5.203201,1.922198,7.581351,5.320124,1.438293
3,ACH-000561,6.037163,1.565597,4.262283,0.790772,1.257011,0.028569,0.056584,0.042644,3.442280,...,3.400538,3.407353,6.154211,4.794936,0.000000,3.984589,1.028569,6.533719,5.132166,2.144046
4,ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.000000,0.042644,4.939227,...,4.125982,4.047015,6.281884,5.853497,0.056584,3.757023,0.056584,5.912171,4.877744,0.815575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,ACH-000242,7.052568,3.094236,3.340562,0.250962,0.097611,0.189034,0.000000,0.344828,4.794416,...,3.933573,3.305971,7.456313,5.276497,0.056584,2.817623,1.722466,6.445429,5.136684,0.137504
557,ACH-000245,5.543496,0.028569,4.438958,0.137504,0.070389,9.429867,0.000000,6.449726,5.584361,...,2.769772,4.072963,6.863195,5.285402,0.000000,0.097611,0.000000,7.263222,6.094025,0.632268
558,ACH-000244,7.407098,2.650765,2.893362,4.918386,0.176323,0.056584,0.014355,0.042644,4.448901,...,4.146492,4.007196,7.645010,3.724650,0.000000,5.080231,0.097611,6.511595,5.846744,3.046142
559,ACH-000247,6.735251,2.899176,4.381975,0.400538,0.070389,0.150560,0.000000,0.028569,4.104337,...,4.138323,3.713696,7.293793,4.537296,0.097611,3.179511,0.659925,5.928370,4.685940,0.604071


In [191]:
gexpr_feature_alt.shape

(561, 698)

In [192]:
# merge these two dataset: a simple merge should work

In [193]:
# methylation and expression data
methylation_and_expr_omics = pd.merge(gexpr_feature_alt, methylation_feature_alt, on = "Cell_line_ID")

In [194]:
methylation_and_expr_omics.head()

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
0,ACH-000828,9.393476,0.042644,3.935460,0.871844,0.070389,0.084064,0.000000,0.056584,3.339137,...,0.52380,0.42795,0.44552,0.23272,0.89944,0.48437,0.24661,0.40100,0.40100,0.36501
1,ACH-000568,7.638074,0.056584,3.427606,0.201634,1.794936,0.739848,0.042644,0.333424,3.193772,...,1.00000,0.36655,0.38361,0.43452,0.88145,0.59839,0.26581,0.30444,0.30444,0.29221
2,ACH-000560,5.728193,6.001352,5.032542,5.018812,0.432959,0.250962,0.000000,0.263034,4.678635,...,0.77780,0.30563,0.33570,0.00000,0.97660,0.12141,0.01751,0.01245,0.01245,0.02566
3,ACH-000561,6.037163,1.565597,4.262283,0.790772,1.257011,0.028569,0.056584,0.042644,3.442280,...,1.00000,0.02078,0.03214,0.01360,0.70737,0.02890,0.00117,0.00069,0.00069,0.01714
4,ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.000000,0.042644,4.939227,...,0.00553,0.00000,0.00000,0.28401,0.61728,0.34487,0.00222,0.00162,0.00162,0.00000


In [195]:
methylation_and_expr_omics.shape

(561, 1506)

In [196]:
# 808+697+1

In [197]:
# sanity check
gexpr_cell_lines = gexpr_feature_alt['Cell_line_ID'].to_list()
methyl_cell_lines = methylation_feature_alt['Cell_line_ID'].to_list()

# Find the intersection using set
intersection = list(set(gexpr_cell_lines) & set(gexpr_cell_lines))
len(intersection)

561

In [198]:
# more sanity checks
gexpr_feature_alt[gexpr_feature_alt['Cell_line_ID'] == 'ACH-000562']

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,NCKIPSD_gexpr,MTCP1_gexpr,DDX3X_gexpr,FANCG_gexpr,SSX2_gexpr,ETV5_gexpr,CEBPA_gexpr,LSM14A_gexpr,CUX1_gexpr,C15orf65_gexpr
4,ACH-000562,7.050502,0.014355,3.360364,0.879706,0.084064,0.137504,0.0,0.042644,4.939227,...,4.125982,4.047015,6.281884,5.853497,0.056584,3.757023,0.056584,5.912171,4.877744,0.815575


In [199]:
methylation_feature_alt[methylation_feature_alt['Cell_line_ID'] == 'ACH-000562']

,Cell_line_ID,SKI_1_2159133_2160133_methyl,TNFRSF14_1_2486803_2487803_methyl,PRDM16_1_2984741_2985741_methyl,RPL22_1_6259679_6260679_methyl,CAMTA1_1_6844383_6845383_methyl,MTOR_1_11322608_11323608_methyl,PRDM2_1_14025734_14026734_methyl,PRDM2_1_14074875_14075875_methyl,CASP9_1_15850940_15851940_methyl,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
369,ACH-000562,0.0,0.02586,0.07884,0.19531,0.0,0.0,0.08649,0.0,0.0,...,0.00553,0.0,0.0,0.28401,0.61728,0.34487,0.00222,0.00162,0.00162,0.0


In [200]:
# Do the same for mutation data
mutation_feature_alt = mutation_feature.reset_index()

In [201]:
mutation_feature_alt = mutation_feature_alt.rename(columns = {'index': 'Cell_line_ID'})

In [202]:
mutation_feature_alt.head()

,Cell_line_ID,TNFRSF14.1:2488170,TNFRSF14.1:2489805,TNFRSF14.1:2489824,TNFRSF14.1:2489856,TNFRSF14.1:2489868,TNFRSF14.1:2491328,TNFRSF14.1:2491335,TNFRSF14.1:2491373,TNFRSF14.1:2491403,...,MTCP1.23:154293902,MTCP1.23:154293909,MTCP1.23:154293951,MTCP1.23:154293978,MTCP1.23:154294034,MTCP1.23:154294203,MTCP1.23:154294207,MTCP1.23:154294278,MTCP1.23:154294279,MTCP1.23:154298967
0,ACH-000828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACH-000568,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACH-000560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACH-000561,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACH-000562,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
mutation_feature_alt.shape

(561, 34674)

In [204]:
# add a suffix '_gexpr' to all columns in the dataset
colnames_with_mutation = [col_name + '_mut' for col_name in mutation_feature_alt.columns[1:]]
cell_line_col_mutation = ['Cell_line_ID']

In [205]:
all_mutation_cols = cell_line_col_mutation + colnames_with_mutation

In [206]:
len(all_mutation_cols)

34674

In [207]:
mutation_feature_alt.columns = all_mutation_cols

In [208]:
mutation_feature_alt.head()

,Cell_line_ID,TNFRSF14.1:2488170_mut,TNFRSF14.1:2489805_mut,TNFRSF14.1:2489824_mut,TNFRSF14.1:2489856_mut,TNFRSF14.1:2489868_mut,TNFRSF14.1:2491328_mut,TNFRSF14.1:2491335_mut,TNFRSF14.1:2491373_mut,TNFRSF14.1:2491403_mut,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
0,ACH-000828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ACH-000568,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ACH-000560,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACH-000561,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACH-000562,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
# Are these all zeros? Hopefully not, let's compute the mean
mutation_feature_alt.mean().mean()

/tmp/ipykernel_2611184/3130069053.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mutation_feature_alt.mean().mean()


0.0013859561753244067

In [210]:
mutation_feature_alt.shape

(561, 34674)

In [211]:
# merge the muation data to the other omics datasets

all_omics_features = pd.merge(methylation_and_expr_omics, mutation_feature_alt, on = "Cell_line_ID")

In [212]:
all_omics_features.shape

(561, 36179)

In [213]:
# sanity check on the number of columns
34673 + 808 + 697 +1

36179

In [214]:
# more sanity checks
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000560'].mean().mean()

/tmp/ipykernel_2611184/2397565948.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000560'].mean().mean()


0.0007787038906353647

In [215]:
all_omics_features[all_omics_features['Cell_line_ID'] == 'ACH-000560'].iloc[:, -34673:].mean().mean()

0.0007787038906353647

In [216]:
# now, merging all the data together
final_deepcdr_data = pd.merge(DeepCDR_data, all_omics_features, on = 'Cell_line_ID', how = 'left')

In [217]:
final_deepcdr_data.head()

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
0,ACH-000070,176870,0.693305,ALL,5.448571,0.000000,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
1,ACH-000137,176870,2.580268,GBM,6.785289,0.422233,3.992768,4.534809,0.189034,0.344828,...,0,0,0,0,0,0,0,0,0,0
2,ACH-000008,176870,2.557837,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
3,ACH-000740,176870,0.290013,HNSC,6.871351,3.005400,4.201634,0.028569,2.301588,0.097611,...,0,0,0,0,0,0,0,0,0,0
4,ACH-000697,176870,1.110250,DLBC,6.501917,0.000000,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0


In [218]:
final_deepcdr_data.shape

(94314, 36182)

In [270]:
############ SANITY CHECKS ###########

In [219]:
# This is the final dataset - Let's do a little more sanity check on it though
ACH_000070_All_data = final_deepcdr_data[final_deepcdr_data['Cell_line_ID'] == 'ACH-000070']

In [220]:
ACH_000070_All_data

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
0,ACH-000070,176870,0.693305,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
166,ACH-000070,65110,6.075025,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
580,ACH-000070,6710780,-5.620091,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
999,ACH-000070,84691,1.324128,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
1418,ACH-000070,6253,-4.928647,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93441,ACH-000070,4261,-2.297562,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
93615,ACH-000070,7251185,-0.590364,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
93789,ACH-000070,462382,-1.703552,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0
93964,ACH-000070,11626927,2.903586,ALL,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,...,0,0,0,0,0,0,0,0,0,0


In [221]:
ACH_000070_All_data.iloc[:, 4:].mean(axis = 0) # loos the same, should be okay

LASP1_gexpr               5.448571
HOXA11_gexpr              0.000000
CREBBP_gexpr              4.661636
ETV1_gexpr                0.056584
GAS7_gexpr                3.068671
                            ...   
MTCP1.23:154294203_mut    0.000000
MTCP1.23:154294207_mut    0.000000
MTCP1.23:154294278_mut    0.000000
MTCP1.23:154294279_mut    0.000000
MTCP1.23:154298967_mut    0.000000
Length: 36178, dtype: float64

In [222]:
# what about the response data? - looks like a match here
DeepCDR_data[DeepCDR_data['Cell_line_ID'] == 'ACH-000070']

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type
0,ACH-000070,176870,0.693305,ALL
166,ACH-000070,65110,6.075025,ALL
580,ACH-000070,6710780,-5.620091,ALL
999,ACH-000070,84691,1.324128,ALL
1418,ACH-000070,6253,-4.928647,ALL
...,...,...,...,...
93441,ACH-000070,4261,-2.297562,ALL
93615,ACH-000070,7251185,-0.590364,ALL
93789,ACH-000070,462382,-1.703552,ALL
93964,ACH-000070,11626927,2.903586,ALL


In [223]:
# Let's separately look at the gene expression, mutation and methylation data

In [227]:
gexpr_feature_alt[gexpr_feature_alt['Cell_line_ID'] == 'ACH-000070']

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,NCKIPSD_gexpr,MTCP1_gexpr,DDX3X_gexpr,FANCG_gexpr,SSX2_gexpr,ETV5_gexpr,CEBPA_gexpr,LSM14A_gexpr,CUX1_gexpr,C15orf65_gexpr
421,ACH-000070,5.448571,0.0,4.661636,0.056584,3.068671,6.994467,0.014355,3.439623,4.858478,...,3.945795,3.789729,6.364572,6.047233,0.014355,6.927185,1.063503,6.772414,4.967169,0.948601


In [229]:
# Extract the columns in the final dataset that corresponds to the gene expression columns only - There's 697 gene expression columns
ACH_000070_All_data_gexpr_columns = ACH_000070_All_data.iloc[:, 4:4+697]

In [232]:
# get the means of these columns and see if they match with above
ACH_000070_All_data_gexpr_columns.mean()

LASP1_gexpr       5.448571
HOXA11_gexpr      0.000000
CREBBP_gexpr      4.661636
ETV1_gexpr        0.056584
GAS7_gexpr        3.068671
                    ...   
ETV5_gexpr        6.927185
CEBPA_gexpr       1.063503
LSM14A_gexpr      6.772414
CUX1_gexpr        4.967169
C15orf65_gexpr    0.948601
Length: 697, dtype: float64

In [233]:
# They match

In [241]:
methylation_feature_alt[methylation_feature_alt['Cell_line_ID'] == 'ACH-000070']

,Cell_line_ID,SKI_1_2159133_2160133_methyl,TNFRSF14_1_2486803_2487803_methyl,PRDM16_1_2984741_2985741_methyl,RPL22_1_6259679_6260679_methyl,CAMTA1_1_6844383_6845383_methyl,MTOR_1_11322608_11323608_methyl,PRDM2_1_14025734_14026734_methyl,PRDM2_1_14074875_14075875_methyl,CASP9_1_15850940_15851940_methyl,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
521,ACH-000070,0.00632,0.0318,0.70146,0.125,0.00551,0.18,0.50138,0.0,0.07479,...,0.16039,0.86472,0.86472,0.16039,0.97643,1.0,0.01051,0.00763,0.00763,0.11111


In [237]:
# Extract the columns in the final dataset that corresponds to the methylation columns only - There's 808 methylation columns
ACH_000070_All_data_methyl_columns = ACH_000070_All_data.iloc[:, 4+697: 4+697+808]

In [238]:
ACH_000070_All_data_methyl_columns.mean()

SKI_1_2159133_2160133_methyl          0.00632
TNFRSF14_1_2486803_2487803_methyl     0.03180
PRDM16_1_2984741_2985741_methyl       0.70146
RPL22_1_6259679_6260679_methyl        0.12500
CAMTA1_1_6844383_6845383_methyl       0.00551
                                       ...   
FLNA_X_153603006_153604006_methyl     1.00000
RPL10_X_153625405_153626405_methyl    0.01051
RPL10_X_153625712_153626712_methyl    0.00763
RPL10_X_153625837_153626837_methyl    0.00763
MTCP1_X_154299547_154300547_methyl    0.11111
Length: 808, dtype: float64

In [239]:
# They match

In [240]:
# Do the same with the mutation data
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000070']

,Cell_line_ID,TNFRSF14.1:2488170_mut,TNFRSF14.1:2489805_mut,TNFRSF14.1:2489824_mut,TNFRSF14.1:2489856_mut,TNFRSF14.1:2489868_mut,TNFRSF14.1:2491328_mut,TNFRSF14.1:2491335_mut,TNFRSF14.1:2491373_mut,TNFRSF14.1:2491403_mut,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
421,ACH-000070,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [254]:
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000070'].values[:,1:].mean()

0.00040377238773685575

In [247]:
# Extract the columns in the final dataset that corresponds to the mutation columns only - There's 34673 mutation columns
ACH_000070_All_data_methyl_columns = ACH_000070_All_data.iloc[:, 4+697+808: 4+697+808+34673]

In [250]:
ACH_000070_All_data_methyl_columns.mean().mean()

0.00040377238773685575

In [ ]:
################### More sanity checks ######################

In [ ]:
# ACH-000008

In [255]:
# More sanity checks - with different IDs
ACH_000008_All_data = final_deepcdr_data[final_deepcdr_data['Cell_line_ID'] == 'ACH-000008']

In [256]:
ACH_000008_All_data

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
2,ACH-000008,176870,2.557837,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
170,ACH-000008,65110,5.794284,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
584,ACH-000008,6710780,-4.709585,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
1003,ACH-000008,84691,1.219886,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
1422,ACH-000008,6253,-0.078097,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93443,ACH-000008,4261,-0.811075,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
93617,ACH-000008,7251185,2.518857,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
93791,ACH-000008,462382,-3.346304,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0
93966,ACH-000008,11626927,4.295117,SKCM,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,...,0,0,0,0,0,0,0,0,0,0


In [257]:
ACH_000008_All_data.iloc[:, 4:].mean(axis = 0) # loos the same, should be okay

LASP1_gexpr               6.312338
HOXA11_gexpr              1.182692
CREBBP_gexpr              3.440952
ETV1_gexpr                4.317594
GAS7_gexpr                7.650836
                            ...   
MTCP1.23:154294203_mut    0.000000
MTCP1.23:154294207_mut    0.000000
MTCP1.23:154294278_mut    0.000000
MTCP1.23:154294279_mut    0.000000
MTCP1.23:154298967_mut    0.000000
Length: 36178, dtype: float64

In [258]:
# what about the response data? - looks like a match here
DeepCDR_data[DeepCDR_data['Cell_line_ID'] == 'ACH-000008']

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type
2,ACH-000008,176870,2.557837,SKCM
170,ACH-000008,65110,5.794284,SKCM
584,ACH-000008,6710780,-4.709585,SKCM
1003,ACH-000008,84691,1.219886,SKCM
1422,ACH-000008,6253,-0.078097,SKCM
...,...,...,...,...
93443,ACH-000008,4261,-0.811075,SKCM
93617,ACH-000008,7251185,2.518857,SKCM
93791,ACH-000008,462382,-3.346304,SKCM
93966,ACH-000008,11626927,4.295117,SKCM


In [223]:
# Let's separately look at the gene expression, mutation and methylation data

In [259]:
gexpr_feature_alt[gexpr_feature_alt['Cell_line_ID'] == 'ACH-000008']

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,NCKIPSD_gexpr,MTCP1_gexpr,DDX3X_gexpr,FANCG_gexpr,SSX2_gexpr,ETV5_gexpr,CEBPA_gexpr,LSM14A_gexpr,CUX1_gexpr,C15orf65_gexpr
402,ACH-000008,6.312338,1.182692,3.440952,4.317594,7.650836,0.176323,0.0,0.263034,4.297925,...,3.950468,4.270529,7.454916,4.781884,5.821965,6.035404,0.0,6.181103,4.084915,1.443607


In [260]:
# Extract the columns in the final dataset that corresponds to the gene expression columns only - There's 697 gene expression columns
ACH_000008_All_data_gexpr_columns = ACH_000008_All_data.iloc[:, 4:4+697]

In [261]:
# get the means of these columns and see if they match with above
ACH_000008_All_data_gexpr_columns.mean()

LASP1_gexpr       6.312338
HOXA11_gexpr      1.182692
CREBBP_gexpr      3.440952
ETV1_gexpr        4.317594
GAS7_gexpr        7.650836
                    ...   
ETV5_gexpr        6.035404
CEBPA_gexpr       0.000000
LSM14A_gexpr      6.181103
CUX1_gexpr        4.084915
C15orf65_gexpr    1.443607
Length: 697, dtype: float64

In [233]:
# They match

In [262]:
methylation_feature_alt[methylation_feature_alt['Cell_line_ID'] == 'ACH-000008']

,Cell_line_ID,SKI_1_2159133_2160133_methyl,TNFRSF14_1_2486803_2487803_methyl,PRDM16_1_2984741_2985741_methyl,RPL22_1_6259679_6260679_methyl,CAMTA1_1_6844383_6845383_methyl,MTOR_1_11322608_11323608_methyl,PRDM2_1_14025734_14026734_methyl,PRDM2_1_14074875_14075875_methyl,CASP9_1_15850940_15851940_methyl,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
290,ACH-000008,0.00046,0.17509,0.06748,0.16667,0.0,0.29591,0.00943,0.0,0.00329,...,1.0,0.51415,0.60747,0.50376,0.63031,0.66341,0.06985,0.02669,0.02669,0.01551


In [263]:
# Extract the columns in the final dataset that corresponds to the methylation columns only - There's 808 methylation columns
ACH_000008_All_data_methyl_columns = ACH_000008_All_data.iloc[:, 4+697: 4+697+808]

In [264]:
ACH_000008_All_data_methyl_columns.mean()

SKI_1_2159133_2160133_methyl          0.00046
TNFRSF14_1_2486803_2487803_methyl     0.17509
PRDM16_1_2984741_2985741_methyl       0.06748
RPL22_1_6259679_6260679_methyl        0.16667
CAMTA1_1_6844383_6845383_methyl       0.00000
                                       ...   
FLNA_X_153603006_153604006_methyl     0.66341
RPL10_X_153625405_153626405_methyl    0.06985
RPL10_X_153625712_153626712_methyl    0.02669
RPL10_X_153625837_153626837_methyl    0.02669
MTCP1_X_154299547_154300547_methyl    0.01551
Length: 808, dtype: float64

In [239]:
# They match

In [266]:
# Do the same with the mutation data
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000008']

,Cell_line_ID,TNFRSF14.1:2488170_mut,TNFRSF14.1:2489805_mut,TNFRSF14.1:2489824_mut,TNFRSF14.1:2489856_mut,TNFRSF14.1:2489868_mut,TNFRSF14.1:2491328_mut,TNFRSF14.1:2491335_mut,TNFRSF14.1:2491373_mut,TNFRSF14.1:2491403_mut,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
402,ACH-000008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000008'].values[:,1:].mean()

0.0005191359270902431

In [268]:
# Extract the columns in the final dataset that corresponds to the mutation columns only - There's 34673 mutation columns
ACH_000008_All_data_methyl_columns = ACH_000008_All_data.iloc[:, 4+697+808: 4+697+808+34673]

In [269]:
ACH_000008_All_data_methyl_columns.mean().mean()

0.0005191359270902431

In [271]:
################### More sanity checks ######################

In [272]:
# ACH-000697

In [273]:
# More sanity checks - with different IDs
ACH_000697_All_data = final_deepcdr_data[final_deepcdr_data['Cell_line_ID'] == 'ACH-000697']

In [274]:
ACH_000697_All_data

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
4,ACH-000697,176870,1.110250,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
174,ACH-000697,65110,5.644170,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
588,ACH-000697,6710780,-4.894291,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
1007,ACH-000697,84691,1.596278,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
1426,ACH-000697,6253,0.657903,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93445,ACH-000697,4261,-1.465742,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
93619,ACH-000697,7251185,0.691604,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
93793,ACH-000697,462382,-0.603306,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0
93968,ACH-000697,11626927,3.287177,DLBC,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,...,0,0,0,0,0,0,0,0,0,0


In [275]:
ACH_000697_All_data.iloc[:, 4:].mean(axis = 0) # looks the same, should be okay

LASP1_gexpr               6.501917
HOXA11_gexpr              0.000000
CREBBP_gexpr              3.341986
ETV1_gexpr                1.594549
GAS7_gexpr                0.084064
                            ...   
MTCP1.23:154294203_mut    0.000000
MTCP1.23:154294207_mut    0.000000
MTCP1.23:154294278_mut    0.000000
MTCP1.23:154294279_mut    0.000000
MTCP1.23:154298967_mut    0.000000
Length: 36178, dtype: float64

In [276]:
# what about the response data? - looks like a match here
DeepCDR_data[DeepCDR_data['Cell_line_ID'] == 'ACH-000697']

,Cell_line_ID,Pubchem_ID,Response,TCGA_cancer_type
4,ACH-000697,176870,1.110250,DLBC
174,ACH-000697,65110,5.644170,DLBC
588,ACH-000697,6710780,-4.894291,DLBC
1007,ACH-000697,84691,1.596278,DLBC
1426,ACH-000697,6253,0.657903,DLBC
...,...,...,...,...
93445,ACH-000697,4261,-1.465742,DLBC
93619,ACH-000697,7251185,0.691604,DLBC
93793,ACH-000697,462382,-0.603306,DLBC
93968,ACH-000697,11626927,3.287177,DLBC


In [223]:
# Let's separately look at the gene expression, mutation and methylation data

In [277]:
gexpr_feature_alt[gexpr_feature_alt['Cell_line_ID'] == 'ACH-000697']

,Cell_line_ID,LASP1_gexpr,HOXA11_gexpr,CREBBP_gexpr,ETV1_gexpr,GAS7_gexpr,CD79B_gexpr,PAX7_gexpr,BTK_gexpr,BRCA1_gexpr,...,NCKIPSD_gexpr,MTCP1_gexpr,DDX3X_gexpr,FANCG_gexpr,SSX2_gexpr,ETV5_gexpr,CEBPA_gexpr,LSM14A_gexpr,CUX1_gexpr,C15orf65_gexpr
435,ACH-000697,6.501917,0.0,3.341986,1.594549,0.084064,1.157044,0.0,0.555816,2.250962,...,3.532317,3.880686,7.581502,3.915521,3.24184,3.430285,0.31034,6.004726,1.641546,1.815575


In [278]:
# Extract the columns in the final dataset that corresponds to the gene expression columns only - There's 697 gene expression columns
ACH_000697_All_data_gexpr_columns = ACH_000697_All_data.iloc[:, 4:4+697]

In [279]:
# get the means of these columns and see if they match with above
ACH_000697_All_data_gexpr_columns.mean()

LASP1_gexpr       6.501917
HOXA11_gexpr      0.000000
CREBBP_gexpr      3.341986
ETV1_gexpr        1.594549
GAS7_gexpr        0.084064
                    ...   
ETV5_gexpr        3.430285
CEBPA_gexpr       0.310340
LSM14A_gexpr      6.004726
CUX1_gexpr        1.641546
C15orf65_gexpr    1.815575
Length: 697, dtype: float64

In [233]:
# They match

In [280]:
methylation_feature_alt[methylation_feature_alt['Cell_line_ID'] == 'ACH-000697']

,Cell_line_ID,SKI_1_2159133_2160133_methyl,TNFRSF14_1_2486803_2487803_methyl,PRDM16_1_2984741_2985741_methyl,RPL22_1_6259679_6260679_methyl,CAMTA1_1_6844383_6845383_methyl,MTOR_1_11322608_11323608_methyl,PRDM2_1_14025734_14026734_methyl,PRDM2_1_14074875_14075875_methyl,CASP9_1_15850940_15851940_methyl,...,BCORL1_X_129138163_129139163_methyl,ELF4_X_129244475_129245475_methyl,ELF4_X_129244688_129245688_methyl,GPC3_X_133119673_133120673_methyl,ATP2B3_X_152800579_152801579_methyl,FLNA_X_153603006_153604006_methyl,RPL10_X_153625405_153626405_methyl,RPL10_X_153625712_153626712_methyl,RPL10_X_153625837_153626837_methyl,MTCP1_X_154299547_154300547_methyl
126,ACH-000697,0.19392,0.13061,0.00331,0.16667,0.00177,0.10105,0.0105,0.00415,0.0,...,0.9444,0.12545,0.13132,0.48962,0.01402,0.47435,0.07883,0.07246,0.07246,0.00404


In [281]:
# Extract the columns in the final dataset that corresponds to the methylation columns only - There's 808 methylation columns
ACH_000697_All_data_methyl_columns = ACH_000697_All_data.iloc[:, 4+697: 4+697+808]

In [282]:
ACH_000697_All_data_methyl_columns.mean()

SKI_1_2159133_2160133_methyl          0.19392
TNFRSF14_1_2486803_2487803_methyl     0.13061
PRDM16_1_2984741_2985741_methyl       0.00331
RPL22_1_6259679_6260679_methyl        0.16667
CAMTA1_1_6844383_6845383_methyl       0.00177
                                       ...   
FLNA_X_153603006_153604006_methyl     0.47435
RPL10_X_153625405_153626405_methyl    0.07883
RPL10_X_153625712_153626712_methyl    0.07246
RPL10_X_153625837_153626837_methyl    0.07246
MTCP1_X_154299547_154300547_methyl    0.00404
Length: 808, dtype: float64

In [283]:
# They match

In [284]:
# Do the same with the mutation data
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000697']

,Cell_line_ID,TNFRSF14.1:2488170_mut,TNFRSF14.1:2489805_mut,TNFRSF14.1:2489824_mut,TNFRSF14.1:2489856_mut,TNFRSF14.1:2489868_mut,TNFRSF14.1:2491328_mut,TNFRSF14.1:2491335_mut,TNFRSF14.1:2491373_mut,TNFRSF14.1:2491403_mut,...,MTCP1.23:154293902_mut,MTCP1.23:154293909_mut,MTCP1.23:154293951_mut,MTCP1.23:154293978_mut,MTCP1.23:154294034_mut,MTCP1.23:154294203_mut,MTCP1.23:154294207_mut,MTCP1.23:154294278_mut,MTCP1.23:154294279_mut,MTCP1.23:154298967_mut
435,ACH-000697,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
mutation_feature_alt[mutation_feature_alt['Cell_line_ID'] == 'ACH-000697'].values[:,1:].mean()

0.0016727713206241166

In [286]:
# Extract the columns in the final dataset that corresponds to the mutation columns only - There's 34673 mutation columns
ACH_000697_All_data_methyl_columns = ACH_000697_All_data.iloc[:, 4+697+808: 4+697+808+34673]

In [287]:
ACH_000697_All_data_methyl_columns.mean().mean()

0.0016727713206241166

In [224]:
# We need to save all the datasets: the final dataset with all features combined, and the individual mutation methylation, and gene expression data